In [1]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Unigrams to Generate a Vocabulary File**

## **Goal**: Make a list of the _n_ most common unigrams for later use filtering multigrams

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

### Download unigrams
Here, I'm using the `download_ngrams` module to fetch unigrams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). Note the `repo_release_id` and `repo_corpus_id` parameters; these tell the module which ngram corpus and release to download.

In [2]:
download_ngram_files(
    ngram_size=1,
    ngram_type='tagged',
    repo_release_id='20200217',
    repo_corpus_id='eng-us',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True
)

Start Time:                2025-02-21 21:10:20.487804

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-us/eng-us-1-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/1download
File index range:          0 to 13
File URLs available:       14
File URLs to use:          14
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng-us/1-00000-of-00014.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-us/1-00013-of-00014.gz
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  True



Downloading:   0%|          | 0/14 [00:00<?, ?files/s]


End Time:                  2025-02-21 21:10:54.214987
Total runtime:             0:00:33.727183



### Convert files from TXT to JSONL
This module converts the original unigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

In [3]:
convert_to_jsonl_files(
    ngram_size=1,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-21 21:11:03.119738

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/2convert
File index range:          0 to 13
Files available:           14
Files to use:              14
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/1download/1-00000-of-00014.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/1download/1-00013-of-00014.txt.lz4
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Converting:   0%|          | 0/14 [00:00<?, ?files/s]


End Time:                  2025-02-21 21:13:03.404228
Total runtime:             0:02:00.284490



### Make unigrams all lowercase
This module lowercases all characters in the unigrams. Most use cases benefit from this.

In [4]:
lowercase_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-21 21:13:23.779161

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/3lowercase
File index range:          0 to 13
Files available:           14
Files to use:              14
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/2convert/1-00000-of-00014.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/2convert/1-00013-of-00014.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lowercasing:   0%|          | 0/14 [00:00<?, ?files/s]


End Time:                  2025-02-21 21:14:30.085361
Total runtime:             0:01:06.306200



### Lemmatize the unigrams
This module lemmatizes the unigrams—that is, reduces them to their base forms. This is desirable for most use cases. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. My code uses the [NLTK Lemmatizer](https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet), which requires requires POS-tagged unigrams. The tags are discarded after lemmatization as they're no longer useful, saving space.

In [5]:
lemmatize_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-21 21:14:49.807204

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/4lemmatize
File index range:          0 to 13
Files available:           14
Files to use:              14
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/3lowercase/1-00000-of-00014.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/3lowercase/1-00013-of-00014.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lemmatizing:   0%|          | 0/14 [00:00<?, ?files/s]


End Time:                  2025-02-21 21:16:08.371058
Total runtime:             0:01:18.563854



### Filter the unigrams
This module removes tokens that provide little information about words' semantic context: those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`).

You can also specify `min_tokens`—the minumum length of a retained ngram after filtering its tokens. This is mainly intended for use when processing multigrams. However, it's still good to specify `min_tokens=1` for unigrams, as it completely discards the data for any unigram violating our criteria. Empty output files will be deleted.

In [6]:
filter_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=1,
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                   2025-02-21 21:16:23.727626

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/5filter
File index range:             0 to 13
Files available:              14
Files to use:                 14
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/4lemmatize/1-00000-of-00014.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/4lemmatize/1-00013-of-00014.jsonl.lz4
Ngram size:                   1
Number of workers:            48
Compress output files:        True
Overwrite existing files:     True
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:          True
Drop ngrams 

Filtering:   0%|          | 0/14 [00:00<?, ?files/s]


Filtering Results (Dropped)
Stopword tokens:              4993 
Short-word tokens:            20751 
Tokens with numerals:         9437345 
Tokens with non-alpha chars:  1388761
Out-of-vocab tokens:          0
Entire ngrams:                10851850 

End Time:                  2025-02-21 21:17:26.506087
Total runtime:             0:01:02.778461



### Sort and combine the unigram files
This modules creates a single, fully-sorted unigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

In [7]:
sort_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    workers=12,
    sort_key='ngram',
    compress=True,
    overwrite=True,
    sort_order='ascending',
    delete_input=True
)

Start Time:                2025-02-21 21:17:38.717811

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-merged.jsonl.lz4
Files available:           10
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/5filter/1-00004-of-00014.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/5filter/1-00013-of-00014.jsonl.lz4
Files to use:              10
Ngram size:                1
Number of workers:         12
Compress output files:     True
Overwrite existing files:  True
Sort key:                  ngram
Sort order:                ascending
Heap-m

Sorting:   0%|          | 0/10 [00:00<?, ?files/s]


Iteration 1: merging 10 files into 5 chunks using 5 workers.
  5 chunk(s) with 2 file(s)

Iteration 2: merging 5 files into 2 chunks using 2 workers.
  1 chunk(s) with 2 file(s)
  1 chunk(s) with 3 file(s)

Iteration 3: final merge of 2 files.


Merging:   0%|          | 0/15441065 [00:00<?, ?lines/s]


Merging complete. Final merged file:
/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-merged.jsonl.lz4

End Time:                  2025-02-21 21:29:33.587902
Total runtime:             0:11:54.870091



### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [8]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/'
        '1gram_files/6corpus/1gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

Lines: 15441065line [03:17, 78062.60line/s]

The file is sorted.


### Consolidate duplicate unigrams
This module consolidates the sorted unigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime with compression:  0:07:33.662163`

In [9]:
consolidate_duplicate_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    lines_per_chunk=500000,
    compress=True,
    overwrite=True
)

Start Time:                2025-02-21 21:33:52.297785

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/temp_chunks
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Created and Sorted: 31 chunks
Merged: 31 chunks


End Time:                  2025-02-21 21:38:34.099404
Total runtime:             0:04:41.801619



### Index unigrams and create vocabulary file
Most use cases will require an indexed list of "valid" (i.e., reasonably common) vocabulary words. This indexing script serves the dual functions of (1) mapping each unigram to an index number (saved in `/6corpus/1gram-corpus-indexed.jsonl`) and (2) culling this file into a vocabulary list consisting of the _n_ most frequent unigrams (saved in `6corpus/1gram-corpus-vocab_list_match.txt`). The vocabulary file provides a critical means of filtering excessively rare words out of the corpus. Unlike files upstream in the workflow, the vocabulary files are not large and don't need to be compressed.

In [10]:
index_and_create_vocab_files(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    vocab_n=80000
)

Start Time:                2025-02-21 21:39:44.379890

Indexing Info
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Indexed file:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-indexed.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Vocabulary Info
Vocab size (top N):        80000
Match File:                /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-corpus-vocab_list_match.txt
Lookup File:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/1gram_files/6corpus/1gram-corpus-vocab_list_lookup.jsonl



Chunking:   0%|          | 0/8221439 [00:00<?, ?lines/s]

Sorting:   0%|          | 0/83 [00:00<?, ?chunks/s]

Merging:   0%|          | 0/8221439 [00:00<?, ?lines/s]

Indexing:   0%|          | 0/8221439 [00:00<?, ?lines/s]


Indexed 8221439 lines.
Final indexed file: 1gram-corpus-indexed.jsonl.lz4
Created vocab_list_match and vocab_list_lookup files for top 80000 ngrams.

End Time:                  2025-02-21 21:47:52.697742
Total runtime:             0:08:08.317852



### Verify indexing [OPTIONAL]
We can verify that the final indexed file looks right.

In [12]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/'
        '1gram_files/6corpus/1gram-corpus-indexed.jsonl.lz4'
    ),
    start_line=1,
    end_line=5,
    parse_json=True
)

Line 1: {'ngram': 'one', 'freq_tot': 2373486232, 'doc_tot': 55886009, 'freq': {'1480': 477, '1489': 295, '1501': 8219, '1506': 48, '1508': 674, '1535': 460, '1539': 433, '1551': 1327, '1553': 439, '1572': 413, '1598': 42, '1601': 5182, '1603': 16, '1612': 240, '1616': 394, '1620': 105, '1622': 165, '1623': 207, '1625': 369, '1626': 532, '1627': 1053, '1634': 976, '1637': 443, '1638': 377, '1642': 477, '1648': 744, '1649': 207, '1655': 245, '1657': 153, '1658': 492, '1659': 540, '1661': 536, '1662': 1124, '1663': 758, '1666': 2838, '1669': 689, '1670': 496, '1671': 786, '1673': 37, '1675': 1111, '1681': 1246, '1682': 787, '1683': 1030, '1689': 526, '1690': 601, '1691': 299, '1693': 3422, '1694': 996, '1700': 6021, '1701': 10951, '1703': 807, '1706': 882, '1707': 575, '1710': 610, '1712': 181, '1716': 3026, '1717': 1588, '1720': 279, '1721': 488, '1723': 781, '1726': 1027, '1729': 1668, '1731': 3521, '1732': 189, '1733': 1498, '1734': 1823, '1735': 1392, '1736': 1093, '1740': 529, '1741'

## Next Steps
If you've gotten this far, you're ready to start pre-processing multigrams using the `workflow_multigrams.ipynb` notebook!